In [4]:
import numpy as np, copy
import matplotlib.pyplot as plt
from astropy import constants as const
import astropy.units as u
from astropy.units import cds
cds.enable()

from scipy.special import expn

from astropy.convolution import convolve, Box1DKernel

#1. Temperature structure of the Sun

In [5]:
data = np.genfromtxt('https://github.com/evansj2014/Astro1Projects/blob/612555ae7452ee107e964f300b2cfe2a4a220639/Data_Files/sun_model.txt', skip_header=24, skip_footer=229, usecols=(1,2,4), names=True)

fig, ax = plt.subplots(1,1, figsize=(6,5))
ax.set_xlim(0,2)
ax.set_ylim(4500,7000)

ax.plot(10**data['lgTauR'], data['T'], label='Sun', lw=5, c="0.75")

Teff = 5777.0
tau = np.linspace(0, 10, 100)

q = 2.0/3.0
T = (0.75*Teff**4*(tau+q))**0.25
ax.plot(tau, T, label='q = 2/3')

q = 0.7104 - 0.133*np.exp(-3.4488*tau)
T = (0.75*Teff**4*(tau+q))*0.25
ax.plot(tau, T, label='numerical q')
ax.legend(loc=0)

ValueError: ignored

#2. Illustration of the integreal in the $C(\alpha, \tau)$ function

#3. Demo of the numpy mesh grid

In [ ]:
x = np.array([1,2,3,4])
y = np.array([10,20])

x_grid, y_grid = np.meshgrid(x, y)

print("The matrices have a shape {}, so [ny, nx]".format(x_grid.shape))

print("Each column of the x matrix has the same value of x:", x_grid)
print("Each row of the y matrix has the same value of y:", y_grid)

print("We can flatten these matrices to forma one-deimensional array:")
x_flat = x_grid.flatten()
y_flat = y_grid.flatten()
print("x: {}".format(x_flat))
print("y: {}".format(y_flat))

print("We cann loop over these new flatten arrays to perform complicated calculations")

Result = np.zeros(x_flat.size)
for i in range(0,Result.size):
  if (x_flat[i] % 2) == 0:
    Result[i] = x_flat[i] + y_flat[i]
  else:
    Result[i] = x_flat[i] - y_flat[i]

print(Result)

print("And then we can reshape our array into a matrix:", Result.reshape(x_grid.shape))

#4. Calculation of the $C(\alpha, \tau)$ function

In [ ]:
n_alpha = 50
n_tau = 5

alpha = np.linspace(0.1, 12, n_alpha)
tau = np.linspace(0, 2, n_tau)

alpha_grid, tau_grid = np.meshgrid(alpha, tau)

alpha_flat = alpha_grid.flatten()
tau_flat = tau_grid.flatten()

C = np.zeros(alpha_flat.size)

def p(tau):
  return 1.0 / (0.75*(tau+2.0/3.0))**0.75

for i in range(0, C.size - 1):
  tt_small = np.linspace(0, tau_flat[i], 1000)
  tt_high = np.linspace(tau_flat[i], 20.0, 1000)

  y = expn(2, tau_flat[i] - tt_small) / (np.exp(p(tt_small)*alpha_flat[i]) - 1) #rest of line cut off think it should be -1
  int_small = np.trapz(y, tt_small)

  y = expn(2, tt_high - tau_flat[i]) / (np.exp(p(tt_high)*alpha_flat[i]) - 1) # rest of line cut off think it should be -1
  int_high = np.trapz(y, tt_high)

  C[i] = int_high - int_small

C = C.reshape(alpha_grid.shape)

print(C)

#5. Let's compare the grey flux with the real Solar flux

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,5))
ax.set_xlim(0, 12)
ax.set_ylim(0, 0.30)

Te = 5777*u.K
wave = np.genfromtxt('/content/drive/MyDrive/Grad Astrophysics 1/sun_wave.txt')
flux = np.genfromtxt('/content/drive/MyDrive/Grad Astrophysics 1/sun_flux.txt')

wave = wave/10*u.nm
alp = (const.h*const.c/wave/const.k_B/Te).decompose()

flux = (flux*10 * u.erg/u.s/u.cm**2/u.nm * const.h * const.c / alp**2 / const.k_B / Te) #rest of line cut off
flux = (flux / const.sigma_sb / (Te**4)).decompose()

ax.plot(alp, convolve(flux, Box1DKernel(31)), c = "0.75", label = "Solar model") #rest of line cut off

ax.legend()